<a href="https://colab.research.google.com/github/nak650228/ITEC/blob/main/VideoRestore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#◢ Video Restoration Project

---


このツールは、Youtubeやローカルファイルから動画をダウンロードし、それをDeepLearningを使って修復します。各処理の修復結果は、ユーザのGoogle Drive上のMovieフォルダに作られます。

このツールでは動画に対して以下の修正を試みます。

　　・DeOldifyによるモノクロ画像のカラー化

　　・Deep Remasterによるノイズ等の除去

　　・Microsoft Bringing-Old-Photos-Back-to-LifeまたはGFPGANによる顔画像の修復  

　　・音声データの抽出し、動画修正時にそれを結合する

　　・オリジナル動画と画像処理後の動画を比較した動画ファイルを作成

　　・TecoGANを使った超高解像度化









#◢ GPUの確認


---


ランタイムに割り当てられたGPUの種類と搭載メモリ容量を確認します。

Tesla K80のように搭載メモリが小さなGPUでは、動画の種類によっては実行にエラーを出す場合があるので注意してください。

In [1]:
#@title 割り当てられたGPUの確認
# Check your current GPU
# If you are lucky, you get 16GB VRAM. If you are not lucky, you get less. VRAM is important. The more VRAM, the higher the maximum resolution will go.

# 16GB: Can handle 720p. 1080p will procude an out-of-memory error. 
# 8GB: Can handle 480p. 720p will produce an out-of-memory error.
import subprocess

!nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv
gpumem=subprocess.run(["nvidia-smi", "--query-gpu=memory.total", "--format=csv,noheader,nounits"], stdout=subprocess.PIPE, text=True)

name, driver_version, memory.total [MiB]
Tesla P100-PCIE-16GB, 460.32.03, 16280 MiB


#◢ 初期設定


---



In [2]:
#@title **フラグの設定と最低限必要なライブラリのインポート**
IS_DEBUG     = True
IS_LIBLOADED = False
IS_MSLOADED  = False
IS_OLDIFYLOADED  = False
IS_GFPGANLOADED  = False
IS_DEEPREMASTERLOADED = False
IS_TECOGANLOADED = False
IS_DAINLOADED = False

!pip install youtube_dl
import youtube_dl
import cv2
import os
import glob
import datetime
import logging
import shutil
from google.colab import files
import moviepy.editor as mpy
from IPython.display import clear_output

     |████████████████████████████████| 1.9 MB 5.0 MB/s 
Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2818048/45929032 bytes (6.1%)6340608/45929032 bytes (13.8%)10084352/45929032 bytes (22.0%)13819904/45929032 bytes (30.1%)17506304/45929032 bytes (38.1%)21266432/45929032 bytes (46.3%)25010176/45929032 bytes (54.5%)28827648/45929032 bytes (62.8%)32587776/45929032 bytes (71.0%)36192256/45929032 bytes (78.8%)39788544/45929032 bytes (86.6%)43

#◢ 動画ファイルのダウンロード


---



In [3]:
#@title **Googleドライブのマウント**
#@markdown Google ドライブを/content/driveにマウントします。
#@markdown このディレクトリの下にMyDrive/Movieというディレクトリがない場合は、それを作ります。

from google.colab import drive
drive.mount('/content/drive', force_remount=True)
print('Google Drive connected.')

if not os.path.isdir("/content/drive/MyDrive/Movie"):
  print("Making directory : /content/drive/MyDrive/Movie")
  os.mkdir("/content/drive/MyDrive/Movie")

Mounted at /content/drive
Google Drive connected.


In [4]:
#@title ##**プロジェクト名の入力** { display-mode: "form" }
#@markdown 任意の名前を入力します。ここで指定した名前のディレクトリ（MyDrive/Movies/指定したプロジェクト名）がGoogle Drive上に作られ、そこに修復された動画がコピーされます。


projectname = 'Gundam2' #@param {type:"string"}

projectDir = '/content/drive/MyDrive/Movie/' + projectname
if os.path.isdir(projectDir):
  print("Warning: " + projectDir + " is still exist.")
  print("Warning: Currnet directory and its contents will be deleted.")

In [5]:
#@title ##**ビデオのダウンロード** { display-mode: "form" }
#@markdown *source_urlフィールドに編集したいビデオのURL（YouTubeやTwitterなど）を入力してください。このフィールドを空白にした場合、ローカルストレージにあるビデオをアップロードできます*
#@markdown *プロジェクト名は任意でOKです。一応最後にGoogle Driveにプロジェクト名で指定したディレクトリが作られて、途中経過も含めて全ての動画ファイルがコピーされます*

%cd /content
! rm -f /content/*.mp4
! rm -f /content/*.aac
! rm -f /content/*.log

# setup logging
logger = logging.getLogger('LoggingTest')
logger.setLevel(10)
fh = logging.FileHandler('/content/restore.log')
logger.addHandler(fh)

formatter = logging.Formatter('%(asctime)s  %(message)s')
fh.setFormatter(formatter)

log_message='Project Name:     '+projectname
logger.info(log_message)

source_url = 'https://www.youtube.com/watch?v=C-KGzjfiRmw' #@param {type:"string"}

if source_url == '':
  dt_start_download = datetime.datetime.now()
  uploaded = files.upload()

  dt_end_download = datetime.datetime.now()
  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))
  os.rename(fn, fn.replace(" ", ""))
  fn = fn.replace(" ", "")
  file_name = "downloaded_video." + fn.split(".")[-1]
  os.rename(fn,file_name)
#  !mv -f $fn $file_name
  log_message='File Name:       '+fn
  logger.info(log_message)

else:
  try:
    ydl_opts = {
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4',
        'outtmpl': 'downloaded_video.mp4',
        }
    dt_start_download = datetime.datetime.now()
    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
      ydl.download([source_url])
    dt_end_download = datetime.datetime.now()
    file_name = 'downloaded_video.mp4'
  
  except BaseException:
    !wget $source_url
    fn = source_url.split('/')[-1]
    os.rename(fn, fn.replace(" ", ""))
    fn = fn.replace(" ", "")
    !mv -f $fn $file_name

  log_message='URL:       '+source_url
  logger.info(log_message)


logger.info('Source file downloaded')

!cp -r /content/downloaded_video.mp4 /content/video.mp4


fps_of_video = int(cv2.VideoCapture(file_name).get(cv2.CAP_PROP_FPS))
frames_of_video = int(cv2.VideoCapture(file_name).get(cv2.CAP_PROP_FRAME_COUNT))
width_of_video = int(cv2.VideoCapture(file_name).get(cv2.CAP_PROP_FRAME_WIDTH))
height_of_video = int(cv2.VideoCapture(file_name).get(cv2.CAP_PROP_FRAME_HEIGHT))
print ("FPS of VIDEO: ",fps_of_video)
print ("Frames of VIDEO: ",frames_of_video)
print ("Width of VIDEO: ",width_of_video)
print ("Height of VIDEO: ",height_of_video)

logger.info('===== Information of video file =====')
log_message='FPS of VIDEO:          '+str(fps_of_video)
logger.info(log_message)
log_message='Frames of VIDEO:       '+str(frames_of_video)
logger.info(log_message)
log_message='Width of VIDEO:        '+str(width_of_video)
logger.info(log_message)
log_message='Height of VIDEO:       '+str(height_of_video)
logger.info(log_message)
logger.info('=====================================')

/content
[youtube] C-KGzjfiRmw: Downloading webpage
[youtube] C-KGzjfiRmw: Downloading player edff9f99
[youtube] C-KGzjfiRmw: Downloading MPD manifest
[download] Destination: downloaded_video.f136.mp4
[download] 100% of 22.50MiB in 05:37
[download] Destination: downloaded_video.mp4.f140
[download] 100% of 1.52MiB in 00:23
[ffmpeg] Merging formats into "downloaded_video.mp4"
Deleting original file downloaded_video.f136.mp4 (pass -k to keep)
Deleting original file downloaded_video.mp4.f140 (pass -k to keep)
FPS of VIDEO:  29
Frames of VIDEO:  2942
Width of VIDEO:  1280
Height of VIDEO:  720


In [6]:
#@title ##**動画の調整** { display-mode: "form" }

#@markdown 　ここではダウンロードした動画にどのような修復を行うかを指定します。

#@markdown　**1. 修復範囲の指定**

#@markdown まず、以下のtarget_start、target_endにダウンロードした動画のどこからどこまでを修復対象にするか、その開始時間と終了時間を指定してください。


target_start = '00:00:29' #@param {type:"string"}
target_end = '00:00:36' #@param {type:"string"}


if os.path.isfile("/content/cropped_video.mp4"):
    os.remove("/content/cropped_video.mp4")

!ffmpeg -i /content/downloaded_video.mp4  -ss $target_start -to $target_end /content/cropped_video.mp4

if os.path.isfile("/content/video.mp4"):
    os.remove("/content/video.mp4")

!cp /content/cropped_video.mp4 /content/video.mp4

##@markdown **インターレスを削除する場合に有効にしてください。**
#is_deinterlace = False #@param {type:"boolean"}

#@markdown　**2. モノクロ動画のカラー化**

#@markdown モノクロ動画にAIで色を付ける場合は、以下のチェックボックスを有効にしてください。
is_deoldify = False #@param {type:"boolean"}


#@markdown レンダリングファクターを指定します(お勧めは10～25）。この値により色の付け方が変わります。
render_factor = 20  #@param {type: "slider", min: 5, max: 44}

#@markdown　**3．ノイズ除去**

#@markdown 以下のチェックボックスを有効にすると、低画質の動画からノイズなどを除去できます。
is_DeepRemaster = True #@param {type:"boolean"}

#@markdown　**4．顔画像の修復**

#@markdown GANを使って、動画の顔部分を書き換えます。この書き換えにはマイクロソフトが開発した 「Bringing-Old-Photos-Back-to-Life」またはテンセントのGFPGANを選択できます。下の入力フィールド右端にある▼をクリックして、どちらを使うか（使わないか）を選択してください。


#@markdown お勧めはマイクロソフトですが、顔の検出能力が低いという欠点があります。GFPGANは顔検出が優秀な反面、元の画質が悪いと生成される画像が作り物っぽくなる傾向があります。 
which_FaceGAN = 'Microsoft' #@param ["None", "GFPGAN", "Microsoft"] {allow-input: true}



#@markdown **5．フレーム補完**

#@markdown 一般的な動画は24～30フレーム/秒ですが、動画によってはフレーム数を落としているものがあります。下のチェックボックスを有効にすると、このような動画のフレーム数を2倍に補完して、動きを滑らかにすることができます。
is_DAIN = True #@param {type:"boolean"}


file_name="/content/cropped_video.mp4"
fps_of_video = int(cv2.VideoCapture(file_name).get(cv2.CAP_PROP_FPS))
frames_of_video = int(cv2.VideoCapture(file_name).get(cv2.CAP_PROP_FRAME_COUNT))
width_of_video = int(cv2.VideoCapture(file_name).get(cv2.CAP_PROP_FRAME_WIDTH))
height_of_video = int(cv2.VideoCapture(file_name).get(cv2.CAP_PROP_FRAME_HEIGHT))

log_message='Start:                       '+target_start
logger.info(log_message)
log_message='End  :                       '+target_end
logger.info(log_message)
log_message='Oldify:                      '+str(is_deoldify)
logger.info(log_message)
log_message='Render Factor:               '+str(render_factor)
logger.info(log_message)
log_message='Deep Remaster:               '+str(is_DeepRemaster)
logger.info(log_message)
log_message='Type of Face GAN:            '+str(which_FaceGAN)
logger.info(log_message)
log_message='DAIN :                       '+str(is_DAIN)
logger.info(log_message)

clear_output()

In [7]:
#@title ##**サイズ調整後の動画を表示（修復対象）** { display-mode: "form" }

#@markdown　ダウンロードした動画から選択された部分を表示します。ここで表示された動画が修復対象となります。

#@markdown  下のフィールド右端の▼をクリックして”download"を選択すると、表示されている動画をダウンロードできます。
what_next = 'play' #@param ["play", "download"]
if what_next == "play":
#  display(mpy.ipython_display("/content/video.mp4", height=400, autoplay=1, maxduration=600))
  display(mpy.ipython_display("/content/video.mp4", autoplay=1, maxduration=600, width=640))
else:
  files.download('/content/video.mp4')

print ("フレームレート: ",fps_of_video)
print ("総フレーム数　: ",frames_of_video)
print ("解像度（横）　: ",width_of_video)
print ("解像度（縦）　: ",height_of_video)

フレームレート:  29
総フレーム数　:  210
解像度（横）　:  1280
解像度（縦）　:  720


#**■メイン処理**
入力が終了したら、この行をクリックした後、Runtimeメニューから「以降のセルを実行」または「Run after」を選択してください。

以降の行が自動的に実行され修復結果がGoogle Drive上にコピーされます。

修復された動画の種類とそのファイル名は以下の通りです。

*   downloaded_video.mp4 　　ダウンロードした動画ファイル

*   cropped_video.mp4 　　　  ダウンロードした動画から修復対象を抜き出したもの

*   upscaled_video.mp4 　　　 cropped_video.mp4の解像度をアップスケールしたもの

*   colorized_video.mp4 　　　 モノクロ動画をカラー化したもの

*   denoised_video.mp4 　　　 ノイズ除去を行ったもの

*   facerestored_video.mp4 　　GANにより顔の修復を行ったもの

*   comparison_output.mp4 　　cropped_video.mp4と修復した動画を比較したもの

*   final_output.mp4 　　　　　全ての修復を行った結果ファイル（音声付き）


#◢ 関数定義


---



In [8]:
#@title 各種関数の定義

def decomposit_video(video_folder,video_file, frame_rate):
  if os.path.isdir(video_folder):
    shutil.rmtree(video_folder)

  os.mkdir(video_folder)

  os.chdir(video_folder)

  decompose_command = 'ffmpeg -i ' + video_file + ' -vf fps=' + str(frame_rate) + ' %09d.png'
  subprocess.run(decompose_command, shell=True)
  #!ffmpeg -i /content/video.mp4 %09d.png

  #clear_output()

def composit_video(video_folder, video_file, frame_rate):
  if os.path.isfile(video_file):
    os.remove(video_file)

  compose_command='ffmpeg -f image2 -framerate ' + str(frame_rate) + ' -i ' + video_folder + '/%09d.png -c:v h264_nvenc -preset slow -qp 18 -pix_fmt yuv420p ' + video_file
  subprocess.run(compose_command, shell=True)
  #clear_output()

def UnSharpMask(image_file, k):
    kernel = np.array([[-k/9.0, -k/9.0, -k/9.0],
                    [-k/9.0, 1 + (8 * k)/9.0, -k/9.0],
                    [-k/9.0, -k/9.0, -k/9.0]])
    dst = cv2.filter2D(image_file, -1, kernel)
    return dst

def make_sharp_kernel(k: int):
  return np.array([
    [-k / 9, -k / 9, -k / 9],
    [-k / 9, 1 + 8 * k / 9, k / 9],
    [-k / 9, -k / 9, -k / 9]
  ], np.float32)

def unsharp_mask_movie(video_folder, video_file, k, frame_rate):
  decomposit_video(video_folder,video_file, frame_rate)

  files = glob.glob(video_folder+'/*.png')
  for ifile in files:
    print("Now processing ",ifile)
    imageblur = cv2.imread(ifile, 0)
#    imagesharp = UnSharpMask(imageblur,k)

    kernel=make_sharp_kernel(1.0)
    imagesharp=cv2.filter2D(imageblur,-1,kernel).astype("uint8")
    cv2.imwrite(ifile, imagesharp)

  composit_video(video_folder, video_file, frame_rate)


from PIL import Image
import numpy as np
import torch
import torch.nn.functional as F
import torchvision

def load_tensor(image_file):
    with Image.open(image_file) as img:
        array = np.asarray(img, np.float32).transpose([2, 0, 1]) / 255.0
        tensor = torch.as_tensor(np.expand_dims(array, axis=0))  # rank 4
    return tensor


#def sharpen_filter(image_file):
#    kernel = np.array([[-2, -2, -2], [-2, 32, -2], [-2, -2, -2]], np.float32) / 16.0  # convolution filter
#    with torch.no_grad():
#        # [out_ch, in_ch, .., ..] : channel wiseに計算
#        sharpen_k = torch.as_tensor(kernel.reshape(1, 1, 3, 3))
#
#        color = load_tensor(image_file)  # color image [1, 3, H, W]
#        # channel-wise conv(大事)　3x3 convなのでPadding=1を入れる
#        multiband = [F.conv2d(color[:, i:i + 1,:,:], sharpen_k, padding=1) for i in range(3)]
#        sharpened_image = torch.cat(multiband, dim=1)
#        torchvision.utils.save_image(sharpened_image, image_file)

def sharpen_filter(image_file, k):
  kernel = np.array([[-float(k)/9.0, -float(k)/9.0, -float(k)/9.0],
                    [-float(k)/9.0, 1 + (8 * float(k))/9.0, -float(k)/9.0],
                    [-float(k)/9.0, -float(k)/9.0, -float(k)/9.0]])
  gray = cv2.imread(image_file,0)
  dst = cv2.filter2D(gray, -1, kernel)
  cv2.imwrite(image_file,dst)
  return dst


def sharpen_movie(video_folder, video_file, k, frame_rate):
  decomposit_video(video_folder,video_file, frame_rate)

  files = glob.glob(video_folder+'/*.png')
  for ifile in files:
    print("Now processing ",ifile)
    sharpen_filter(ifile, k)

  composit_video(video_folder, video_file, frame_rate)


#def sharpen_movie(video_folder, video_file, k, frame_rate):
#  decomposit_video(video_folder,video_file, frame_rate)
#
#  files = glob.glob(video_folder+'/*.png')
#  for ifile in files:
#    print("Now processing ",ifile)
#    sharpen_filter(ifile)

#  composit_video(video_folder, video_file, frame_rate)


def hist_filter(image_file):
    img = cv2.imread(image_file,0)

# create a CLAHE object (Arguments are optional).
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    cl1 = clahe.apply(img)

    cv2.imwrite(image_file,cl1)


def changehistgram_movie(video_folder, video_file, frame_rate):
  decomposit_video(video_folder,video_file, frame_rate)

  files = glob.glob(video_folder+'/*.png')
  for ifile in files:
    print("Now processing ",ifile)
    hist_filter(ifile)

  composit_video(video_folder, video_file, frame_rate)

def colorize_video(source_video, output_video, render_factor):
  print(source_video)
  print(output_video)

  if os.path.isfile("/content/DeOldify/video"):
    shutil.rmtree("/content/DeOldify/video")

  !mkdir -p '/content/DeOldify/video/source'

  command1="cp -r " + source_video + " /content/DeOldify/video/source/video.mp4"
  subprocess.run(command1,shell=True)
  print(command1)
#  !cp -r str(source_video) /content/DeOldify/video/source/video.mp4
  video_path = colorizer.colorize_from_file_name('/content/DeOldify/video/source/video.mp4', render_factor)
 
  command2="cp -r /content/DeOldify/video/result/video.mp4 " + output_video
  subprocess.run(command2,shell=True)
#  !cp -r /content/DeOldify/video/result/video.mp4 str(output_video)
  print(command2)
  if os.path.isfile("/content/DeOldify/video/result/video.mp4"):
    os.remove("/content/DeOldify/video/result/video.mp4")


def clear_logs(flag):
  if flag != True:
    clear_output()


!mkdir /content/reference



In [9]:
#@title ライブラリ等のインストール

if IS_LIBLOADED != True:

  %cd /content

  p = os.getenv('PATH')
  ld = os.getenv('LD_LIBRARY_PATH')
  os.environ['PATH'] = f"/usr/local/cuda-10.1/bin:{p}"
  os.environ['LD_LIBRARY_PATH'] = f"/usr/local/cuda-10.1/lib64:{ld}"

  !pip install ffmpeg
  !pip install ffmpeg-python
  !pip install torch==1.10.0
  !pip install torchvision==0.10.1
  !pip install scipy
  !pip install imgaug

  !pip install tensorflow
  #!pip install tensorflow==1.15.5
  #!pip install imgaug==0.2.5
  #!pip install scipy==1.2.0
  #!pip install torch==1.4
  #!pip install torchvision==0.5

  #シーン分割
  #!pip install scenedetect[opencv,progress_bar]
  #!pip install subprocess

  import subprocess
  import tensorflow as tf
  import ffmpeg
  import numpy as np
  import imageio
  import cv2
  import torch
  import glob
  import shutil
  import time
  from IPython.display import clear_output


  torch.backends.cudnn.benchmark=True

#clear_output()
  IS_LIBLOADED = True

  clear_logs(IS_DEBUG)

/content
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6083 sha256=fff57bd5a0676ffbf8c1a8880fbbca919bb8bf195c4ff4248033ee3cff5afe3b
  Stored in directory: /root/.cache/pip/wheels/64/80/6e/caa3e16deb0267c3cbfd36862058a724144e19fdb9eb03af0f
Successfully built ffmpeg
     |████████████████████████████████| 22.1 MB 1.3 MB/s 
     |████████████████████████████████| 831.4 MB 6.5 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.11.1+cu111
    Uninstalling torchvision-0.11.1+cu111:
      Successfully uninstalled torchvision-0.11.1+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.11.0 requires torch==1.10.0, but you have tor

# ◢ 音声情報の抽出

---



In [10]:
#@title 音声を抽出し、output.aacという名前で保存
if os.path.isfile("/content/output.aac"):
  os.remove("/content/output.aac")

%cd /content/

!ffmpeg -i /content/cropped_video.mp4 -vn -y -acodec copy output.aac
clear_logs(IS_DEBUG)

/content
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --e

# ◢ TecoGANによる動画のアップスケール


---



In [11]:
#@title ##**TecoGANのリポジトリをクローンする** { display-mode: "form" }

if IS_TECOGANLOADED != True :
  log_message='Start cloning TecoGAN Repository'
  logger.info(log_message)

  %cd /content
  !git clone https://github.com/JoeyBallentine/Video-Inference.git
  !git clone https://github.com/skycrapers/TecoGAN-PyTorch.git

  %cd /content/TecoGAN-PyTorch/
  !bash /content/TecoGAN-PyTorch/scripts/download/download_models.sh BD TecoGAN
  !bash /content/TecoGAN-PyTorch/scripts/download/download_models.sh BI TecoGAN
  !cp /content/TecoGAN-PyTorch/pretrained_models/* /content/Video-Inference/models/

  IS_TECOGANLOADED = True

  log_message='Finished cloning TecoGAN Repository'
  logger.info(log_message)

  clear_output()

In [12]:
#@title ##**動画のアップスケール** { display-mode: "form" }
if os.path.isfile("/content/upscaled_video.mp4"):
  os.remove("/content/upscaled_video.mp4")


if height_of_video <= 720: 
  log_message='Start upscaling with TecoGAN'
  logger.info(log_message)

  %cd /content/Video-Inference/
  if os.path.isfile("./output/video.mp4"):
      os.remove("./output/video.mp4")

  !python run.py ./models/TecoGAN_BD_iter500000.pth --input "/content/video.mp4" --output "/content/Video-Inference/output/video.mp4"

  if (height_of_video * 4) >= 1080:
    !ffmpeg -i /content/Video-Inference/output/video.mp4 -vf scale=-1:1080 /content/upscaled_video.mp4
  else:
    !cp ./output/video.mp4 /content/upscaled_video.mp4

  !cp /content/upscaled_video.mp4 /content/video.mp4

  log_message='Finish upscaling with TecoGAN'
  logger.info(log_message)

  clear_logs(IS_DEBUG)

/content/Video-Inference
100% (210 of 210) |#######################| Elapsed Time: 0:03:28 Time:  0:03:28
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-l

# ◢ DeOldifyによるモノクロ動画への彩色


---



In [13]:
#@title **DeOldifyのリポジトリをクローンする**
if IS_OLDIFYLOADED != True and  is_deoldify == True:
  log_message='Start cloning DeOldify Repository'
  logger.info(log_message)

  %cd /content

  !git clone https://github.com/jantic/DeOldify.git DeOldify
  %cd DeOldify

  #NOTE:  This must be the first call in order to work properly!
  from deoldify import device
  from deoldify.device_id import DeviceId
  #choices:  CPU, GPU0...GPU7
  device.set(device=DeviceId.GPU0)

  import torch

  if not torch.cuda.is_available():
    print('GPU not available.')

  from os import path

  !pip install -r colab_requirements.txt

  import fastai
  from deoldify.visualize import *
  from pathlib import Path
  torch.backends.cudnn.benchmark=True
  import warnings
  warnings.filterwarnings("ignore", category=UserWarning, message=".*?Your .*? set is empty.*?")

  !mkdir 'models'
  !wget https://data.deepai.org/deoldify/ColorizeVideo_gen.pth -O ./models/ColorizeVideo_gen.pth

  colorizer = get_video_colorizer()

  IS_OLDIFYLOADED = True

  log_message='Finish cloning DeOldify Repository'
  logger.info(log_message)
  clear_output()

In [14]:
#@title **Deoldifyによるモノクロ動画のカラー化を行う**
if is_deoldify == True:
  log_message='Start colorization with DeOldify'
  logger.info(log_message)

  %cd /content/DeOldify

  if os.path.isfile("/content/DeOldify/video"):
    shutil.rmtree("/content/DeOldify/video")

  !mkdir -p '/content/DeOldify/video/source'

  !cp -r /content/video.mp4 /content/DeOldify/video/source/video.mp4
  video_path = colorizer.colorize_from_file_name('/content/DeOldify/video/source/video.mp4', render_factor)
  !cp -r /content/DeOldify/video/result/video.mp4 /content/colorized_video.mp4
  !cp -r /content/colorized_video.mp4 /content/video.mp4
  if os.path.isfile("/content/DeOldify/video/result/video.mp4"):
    os.remove("/content/DeOldify/video/result/video.mp4")

  log_message='Finish colorization with DeOldify'
  logger.info(log_message)

#colorize_video('/content/video.mp4', '/content/colorized_video.mp4', render_factor)
#!cp /content/colorized_video /content/video.mp4
clear_logs(IS_DEBUG)

# ◢ ノイズ除去(Deep Remaster)



---



In [15]:
#@title ##**DeepRemasterのリポジトリをクローンする** { display-mode: "form" }

if IS_DEEPREMASTERLOADED != True and is_DeepRemaster == True:
  log_message='Start cloning Deep Remaster Repository'
  logger.info(log_message)

  %cd /content
  !git clone https://github.com/satoshiiizuka/siggraphasia2019_remastering.git DeepRemaster
  !cp -r /content/video.mp4 /content/DeepRemaster/
  %cd /content/DeepRemaster

  !wget --continue -O model/remasternet.pth.tar -- http://iizuka.cs.tsukuba.ac.jp/data/remasternet.pth.tar

  log_message='Finish cloning Deep Remaster Repository'
  logger.info(log_message)

  IS_DEEPREMASTERLOADED = True
  clear_output()

In [ ]:
#@title ##**画像からノイズを除去する** { display-mode: "form" }
if os.path.isfile("/content/denoised_video.mp4"):
  os.remove("/content/denoised_video.mp4")


if is_DeepRemaster == True:

  log_message='Start denoising with Deep Remaster'
  logger.info(log_message)

  %cd /content/DeepRemaster


  #command = "python remaster.py --input /content/video.mp4 --disable_colorization --gpu --mindim " + str(int(height_of_video) * 2)
  !python remaster.py --input /content/video.mp4 --disable_colorization --gpu --mindim 1080

  #subprocess.run(command,shell=True)

  !cp /content/video.mp4 /content/denoised_video.mp4
#!python remaster.py --input /content/video.mp4 --disable_colorization --gpu --mindim 1080
#!python remaster.py --input /content/video.mp4 --disable_colorization --gpu
#clear_output()

  log_message='Finish denoising with Deep Remaster'
  logger.info(log_message)

clear_logs(IS_DEBUG)

/content/DeepRemaster
Processing video.mp4...
 55% 115/210 [05:32<04:10,  2.64s/it]

#◢ Microsoft Bringing-Old-Photos-Back-to-Lifeによる画像修正


---



In [ ]:
#@title **Microsoft Bringing-Old-Photos-Back-to-Lifeのリポジトリーをクローンする**

if IS_MSLOADED != True and which_FaceGAN == "Microsoft":

  log_message='Start cloning Microsoft Bringing-Old-Photos-Back-to-Life repository'
  logger.info(log_message)

  %cd /content
  !git clone https://github.com/microsoft/Bringing-Old-Photos-Back-to-Life.git photo_restoration

  #@markdown Microsoft Bringing Old-Photos-Back-to-Lifeの学習済みモデルをダウンロード
  # pull the syncBN repo
  %cd photo_restoration/Face_Enhancement/models/networks
  !git clone https://github.com/vacancy/Synchronized-BatchNorm-PyTorch
  !cp -rf Synchronized-BatchNorm-PyTorch/sync_batchnorm .
  %cd ../../../

  %cd Global/detection_models
  !git clone https://github.com/vacancy/Synchronized-BatchNorm-PyTorch
  !cp -rf Synchronized-BatchNorm-PyTorch/sync_batchnorm .
  %cd ../../

  # download the landmark detection model
  %cd Face_Detection/
  !wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
  !bzip2 -d shape_predictor_68_face_landmarks.dat.bz2
  %cd ../

  # download the pretrained model
  %cd Face_Enhancement/
  !wget https://facevc.blob.core.windows.net/zhanbo/old_photo/pretrain/Face_Enhancement/checkpoints.zip
  !unzip -o checkpoints.zip
  %cd ../

  %cd Global/
  !wget https://facevc.blob.core.windows.net/zhanbo/old_photo/pretrain/Global/checkpoints.zip
  !unzip -o checkpoints.zip
  %cd ../

  ! pip install -r requirements.txt
  
  log_message='Finish cloning Microsoft Bringing-Old-Photos-Back-to-Life repository'
  logger.info(log_message)

  IS_MSLOADED = True

  clear_output()

In [ ]:
#@title **顔画像の修復**
# ffmpeg extract - Generating individual frame PNGs from the source file.

if os.path.isfile("/content/facerestored_video.mp4"):
  os.remove("/content/facerestored_video.mp4")

if which_FaceGAN == "Microsoft":
  log_message='Start face restoration with Microsoft Bringing-Old-Photos-Back-to-Life'
  logger.info(log_message)

  %cd /content/photo_restoration

  FRAME_INPUT_DIR = "/content/photo_restoration/input_frames"
  FRAME_OUTPUT_DIR = "/content/photo_restoration/output_frames"
  INPUT_FILEPATH = "/content/video.mp4"

  if os.path.isfile(FRAME_INPUT_DIR):
     shutil.rmtree(FRAME_INPUT_DIR)
  %shell mkdir -p '{FRAME_INPUT_DIR}'
       
  command = "ffmpeg -i " + INPUT_FILEPATH + " -vf framerate=" + str(fps_of_video) + " " +  FRAME_INPUT_DIR + "/%05d.png"
  subprocess.run(command,shell=True)

#  %shell ffmpeg -i '{INPUT_FILEPATH}' '{FRAME_INPUT_DIR}/%05d.png'

  if os.path.isfile(FRAME_OUTPUT_DIR):
    shutil.rmtree(FRAME_OUTPUT_DIR)
  
  %shell mkdir -p '{FRAME_OUTPUT_DIR}'


  png_generated_count_command_result = %shell ls '{FRAME_INPUT_DIR}' | wc -l
  from IPython.display import clear_output

  pngs_generated_count = int(png_generated_count_command_result.output.strip())


  #print(f"Input FPS: {fps}")
  print(f"{pngs_generated_count} frame PNGs generated.")

  # Checking if PNG do have alpha
  import subprocess as sp
  %cd {FRAME_INPUT_DIR}
  channels = sp.getoutput('identify -format %[channels] 00001.png')
  print (f"{channels} detected")

  # Removing alpha if detected
  if "a" in channels:
    print("Alpha detected and will be removed.")
    print(sp.getoutput('find . -name "*.png" -exec convert "{}" -alpha off PNG24:"{}" \;'))

  %cd /content/photo_restoration
  input_folder = FRAME_INPUT_DIR
  output_folder = FRAME_OUTPUT_DIR

  !rm -rf /content/photo_restoration/output_frames/*

  print (input_folder)
  print (output_folder)

  import os
  basepath = os.getcwd()
  #input_path = os.path.join(basepath, input_folder)
  #output_path = os.path.join(basepath, output_folder)
  #os.mkdir(output_path)
  #!rm -rf output_folder
  #os.mkdir(output_folder)

  !python run.py --HR --input_folder /content/photo_restoration/input_frames --output_folder /content/photo_restoration/output_frames --GPU 0

#create video

  %cd /content/photo_restoration/output_frames/final_output
  command="ffmpeg  -pattern_type glob -i '*.png' -c:v h264_nvenc -pix_fmt yuv420p -framerate " + str(fps_of_video) + " /content/facerestored_video.mp4"
  #!ffmpeg  -pattern_type glob -i '*.png' -c:v h264_nvenc -pix_fmt yuv420p /content/facerestored_video.mp4
  subprocess.run(command,shell=True)
  !rm -f /content/video.mp4
  !cp /content/facerestored_video.mp4 /content/video.mp4
  #clear_output()

  log_message='Finish face restoration with Microsoft Bringing-Old-Photos-Back-to-Life'
  logger.info(log_message)

  clear_logs(IS_DEBUG)

In [ ]:
#@title **フレームレートの調整**
# Microsoft Bringing-Old-Photos-Back-to-Life では動画をデフォルトで25FPSで作ってしまうため、オリジナルのFPSに変換
if which_FaceGAN == "Microsoft":
  decomposit_video("/content/datas","/content/facerestored_video.mp4", 25)
  
  composit_video("/content/datas","/content/adjusted_video.mp4",fps_of_video)

  !cp /content/adjusted_video.mp4 /content/video.mp4
  !rm -f /content/adjusted_video.mp4
  
  clear_logs(IS_DEBUG)

#◢ GFPGANによる顔画像の修復


---



In [ ]:
#@title ##**GFPGANのリポジトリをクローンする**
if IS_GFPGANLOADED != True and which_FaceGAN == "GFPGAN":

  log_message='Start cloning GFPGAN repository'
  logger.info(log_message)

  # Clone GFPGAN and enter the GFPGAN folder
  %cd /content
  !rm -rf GFPGAN
  !git clone https://github.com/TencentARC/GFPGAN.git
  %cd GFPGAN

  # Set up the environment
  # Install basicsr - https://github.com/xinntao/BasicSR
  # We use BasicSR for both training and inference
  !pip install basicsr
  # Install facexlib - https://github.com/xinntao/facexlib
  # We use face detection and face restoration helper in the facexlib package
  !pip install facexlib
  # Install other depencencies
  !pip install -r requirements.txt
  !python setup.py develop
  !pip install realesrgan  # used for enhancing the background (non-face) regions
  # Download the pre-trained model
  !wget https://github.com/TencentARC/GFPGAN/releases/download/v0.2.0/GFPGANCleanv1-NoCE-C2.pth -P experiments/pretrained_models
  
  log_message='Stop cloning GFPGAN repository'
  logger.info(log_message)

  IS_GFPGANLOADED = True
  
  clear_output()

In [ ]:
#@title ##**GFPGANを使って動画ファイル中の顔画像を修復** { display-mode: "form" }

if os.path.isfile("/content/facerestored_video.mp4") :
  !rm -f /content/facerestored_video.mp4

if which_FaceGAN == "GFPGAN":
  log_message='Start face restoration with GFPGAN'
  logger.info(log_message)

  !pip install torch==1.10.0 
  !pip install torchvision==0.10.1

  import os
  p = os.getenv('PATH')
  ld = os.getenv('LD_LIBRARY_PATH')
  os.environ['PATH'] = f"/usr/local/cuda-10.1/bin:{p}"
  os.environ['LD_LIBRARY_PATH'] = f"/usr/local/cuda-10.1/lib64:{ld}"

  upload_folder = "/content/GFPGAN/inputs/upload"
  if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)

  os.mkdir(upload_folder)

  %cd /content/GFPGAN/inputs/upload

  !ffmpeg -i /content/video.mp4 %09d.png

# Now we use the GFPGAN to restore the above low-quality images
# We use [Real-ESRGAN](https://github.com/xinntao/Real-ESRGAN) for enhancing the background (non-face) regions

  %cd /content/GFPGAN
  !rm -rf results
  !python inference_gfpgan.py --upscale 2 --test_path inputs/upload --save_root results --model_path experiments/pretrained_models/GFPGANCleanv1-NoCE-C2.pth --bg_upsampler realesrgan


  command="ffmpeg -f image2 -framerate " + str(fps_of_video) + " -i /content/GFPGAN/results/restored_imgs/%09d.png -c:v h264_nvenc -preset slow -qp 18 -pix_fmt yuv420p /content/facerestored_video.mp4 "
  subprocess.run(command,shell=True)
  #!ffmpeg -f image2 -framerate 30 -i /content/GFPGAN/results/restored_imgs/%09d.png -c:v h264_nvenc -preset slow -qp 18 -pix_fmt yuv420p /content/facerestored_video.mp4
  !rm -f /content/video.mp4
  !cp /content/facerestored_video.mp4 /content/video.mp4

  log_message='Finish face restoration with GFPGAN'
  logger.info(log_message)
 
  clear_logs(IS_DEBUG)

# ◢ DAINによるフレーム補完


---




In [ ]:
#@title ##**DAINのリポジトリをクローンする** { display-mode: "form" }

if IS_DAINLOADED != True and is_DAIN == True:

  log_message='Start cloning DAIN repository'
  logger.info(log_message)

  %cd /content
  !git clone https://github.com/HeylonNHP/Dain-App.git
  !pip install ninja
 # !wget https://developer.download.nvidia.com/compute/cuda/10.2/Prod/local_installers/cuda-repo-ubuntu1804-10-2-local-10.2.89-440.33.01_1.0-1_amd64.deb
 # !dpkg -i cuda-repo-ubuntu1804-10-2-local-10.2.89-440.33.01_1.0-1_amd64.deb

  import os
  p = os.getenv('PATH')
  ld = os.getenv('LD_LIBRARY_PATH')
  os.environ['PATH'] = f"/usr/local/cuda-10.1/bin:{p}"
  os.environ['LD_LIBRARY_PATH'] = f"/usr/local/cuda-10.1/lib64:{ld}"


  %cd /content/Dain-App/my_package
  !sh build.sh
  %cd /content/Dain-App/PWCNet/correlation_package_pytorch1_0
  !sh build.sh

  !pip install PyQt5
  #!sudo add-apt-repository -y ppa:jonathonf/ffmpeg-4
  #!sudo apt update
  #!sudo apt -y install ffmpeg

  log_message='Stop cloning DAIN repository'
  logger.info(log_message)

  IS_DAINLOADED = True

  clear_output()

In [ ]:
#@title ##**DAINによるフレーム補完の実行** { display-mode: "form" }
%cd /content

if os.path.isfile("/content/interpolated_video.mp4"):
  os.remove("/content/interpolated_video.mp4")

if is_DAIN == True:

  log_message='Start frame interporation with DAIN'
  logger.info(log_message)


  %cd /content/Dain-App
  !rm -rf /content/Dain-App/example_folder/output_videos/*.mp4

  ##!python  my_design.py -cli -h
  !cp /content/video.mp4 /content/Dain-App/video.mp4

  !python  my_design.py -cli --input "/content/Dain-App/video.mp4" -o "/content/Dain-App/example_folder/" -on "/content/interpolated_video.mp4" -m "model_weights/best.pth" -fh 2 --interpolations 2 --depth_awarenes 0 --loop 0 -p 0 --alpha 0 --check_scene_change 10 --png_compress 0 --crf 1 --pixel_upscale_downscale_before 1 --pixel_downscale_upscale_after 1 --pixel_upscale_after 1 --mute_ffmpeg 0 --split_size_x -1 --split_size_y -1 --split_pad 150 --half 0 --step_extract 1 --step_interpolate 1 --batch_size 1 --use_benchmark 0 --force_flow 1 --smooth_flow 0 --downscale -1 --fast_mode 0 -cif 1 -cof 1 -cc 1 -csc 1
  ##!python  my_design.py -cli --input "/content/Dain-App/video.mp4" -o "/content/Dain-App/example_folder/" -on "interpolated.mp4" -m "model_weights/best.pth" -fh 2 --interpolations 2 --depth_awarenes 0 --loop 0 -p 0 --alpha 0 --check_scene_change 10 --png_compress 0 --crf 1 --pixel_upscale_downscale_before 1 --pixel_downscale_upscale_after 1 --pixel_upscale_after 1 --mute_ffmpeg 0 --split_size_x -1 --split_size_y -1 --split_pad 150 --half 0 --step_extract 1 --step_interpolate 1 --batch_size 1 --use_benchmark 0 --force_flow 1 --smooth_flow 0 --downscale -1 --fast_mode 0
  !cp /content/Dain-App/example_folder/output_videos/*.mp4 /content/interpolated_video.mp4
  !cp /content/interpolated_video.mp4 /content/video.mp4

  log_message='Finish frame interporation with DAIN'
  logger.info(log_message)

  clear_logs(IS_DEBUG)

#◢ 最終処理


---



In [ ]:
#@title 音声のアペンド
%cd /content

if os.path.isfile("/content/final_output.mp4"):
  os.remove("/content/final_output.mp4")
  
!ffmpeg -i /content/video.mp4 -i /content/output.aac -c:v copy -c:a aac /content/final_output.mp4
#!cp /content/final_output.mp4 /content/video.mp4

clear_logs(IS_DEBUG)

In [ ]:
#@title 比較動画(comparison_output.mp4)の作成
if os.path.isfile("/content/comparison_output.mp4"):
  os.remove("/content/comparison_output.mp4")

if is_DAIN != True:
  log_message='Start making comparison video'
  logger.info(log_message)

  !ffmpeg -i /content/cropped_video.mp4 \
       -i /content/final_output.mp4 \
       -filter_complex "[0:v]scale=720:-2[v0];[1:v]scale=720:-2[v1];[v0][v1]hstack=inputs=2" \
       -vcodec libx264 -crf 23 /content/comparison_output.mp4

  log_message='Finish making comparison video'
  logger.info(log_message)

  clear_logs(IS_DEBUG)

In [ ]:
#@title ##**Google Driveへのファイル・バックアップ** { display-mode: "form" }
log_message='Start copying results in Google Drive'
logger.info(log_message)

if os.path.isdir(projectDir):
    shutil.rmtree(projectDir)

os.mkdir(projectDir)
os.chdir(projectDir)
!cp /content/*.mp4 .
!cp /content/output.aac .

!cp /content/restore.log .

log_message='Finish copying results in Google Drive'
logger.info(log_message)

clear_logs(IS_DEBUG)

#◢ 修復結果の確認


---



In [ ]:
#@title ##**修復した動画を表示** { display-mode: "form" }

what_next = 'play' #@param ["play", "download"]
if what_next == "play":
#  display(mpy.ipython_display("/content/video.mp4", height=400, autoplay=1, maxduration=600))
  display(mpy.ipython_display("/content/final_output.mp4", autoplay=1, maxduration=600, width=640))
else:
  files.download('/content/video.mp4')
